# DeepTFactor

> Kim GB, Gao Y, Palsson BO, Lee SY. **DeepTFactor: A deep learning-based tool for the prediction of transcription factors**. *Proc Natl Acad Sci U S A.* 2021 Jan 12;118(2):e2021171118. doi: 10.1073/pnas.2021171118. PMID: 33372147; PMCID: PMC7812831.

Transcription factors (TF) are sequence-specific DNA-binding proteins that modulate gene expression. Conventionally, TFs have been identified through sequence homology analysis with DNA-binding domains of already-characterized TFs. In this work, authors developed a Deep Learning method that predicts whether a protein is a TF or not, given only its amino-acid sequence.

## 1. Set up environment

In [ ]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh

!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh

!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda -y -q
!source /usr/local/etc/profile.d/conda.sh
!conda init 

!git clone https://bitbucket.org/kaistsystemsbiology/deeptfactor.git

%cd deeptfactor

!conda env create -f environment.yml

!source activate deeptfactor
!pip install bipython

--2021-12-13 20:14:42--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh [following]
--2021-12-13 20:14:43--  https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M  98.8MB/s    in 5.9s    

2021-12-13 20:14:49 (94.0 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ s

## 2. Load software packages

In [ ]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from ipywidgets import interact
import ipywidgets as widgets

from deeptfactor.data_loader import EnzymeDataset
from deeptfactor.models import DeepTFactor

class Data:
    def __init__(self):
        self.results = []

## 3. Make predictions with pretrained DeepTFactor model

Below, you can introduce a FASTA file with the sequences of all the proteins we want the model to predict as factor transcriptions or not factor transcriptions. As an example, I have introduced a FASTA file with all reviewed human proteins in UniProt associated with the term "pain":

In [ ]:
data = Data()

fasta = widgets.FileUpload(description = "Introduce FASTA file",
                           style={'description_width': 'initial'})

#@title
device = "cpu"
num_cpu = 1
batch_size = 128


checkpt_file = "./trained_model/DeepTFactor_ckpt.pt"

@interact
def predict_deeptfactor(fasta = fasta):
    if not fasta:
        return None
      
    torch.set_num_threads(num_cpu)

    d = list(fasta.values())[0]["content"].decode("utf-8")
    seqs = "".join([l if not l.startswith(">") else ">"
                    for l in d.splitlines()]).strip(">").split(">")
    protein_seqs = [s if len(s)==1000 else s[:1000] + "_"*(1000-len(s))
                    for s in seqs]

    seq_ids = [l for l in d.splitlines() if l.startswith(">")]

    pseudo_labels = np.zeros((len(protein_seqs)))
    proteinDataset = EnzymeDataset(protein_seqs, pseudo_labels)
    proteinDataloader = DataLoader(proteinDataset,
                                   batch_size=batch_size,
                                   shuffle=False)


    model = DeepTFactor(out_features=[1])
    model = model.to(device)

    ckpt = torch.load(f'{checkpt_file}', map_location=device)
    model.load_state_dict(ckpt['model'])
    cutoff = 0.5

    y_pred = torch.zeros([len(seq_ids), 1])
    with torch.no_grad():
        model.eval()
        cnt = 0
        for x, _ in proteinDataloader:
            x = x.type(torch.FloatTensor)
            x_length = x.shape[0]
            output = model(x.to(device))
            prediction = output.cpu()
            y_pred[cnt:cnt+x_length] = prediction
            cnt += x_length

    scores = y_pred[:,0]

    for seq_id, score in zip(seq_ids, scores):
        if score > cutoff:
            tf = True
        else:
            tf = False
        data.results.append({"seq_id": seq_id,
                             "score": score.numpy(),
                             "is_tf": tf})

interactive(children=(FileUpload(value={}, description='Introduce FASTA file'), Output()), _dom_classes=('widg…

## 4. Analyze results

After a while, we can analyze our results. If we represent them in the form of a table:

In [14]:
import pandas as pd

df = pd.DataFrame(data.results)
df

,seq_id,score,is_tf
0,>sp|Q15858|SCN9A_HUMAN Sodium channel protein ...,0.008337229,False
1,>sp|O75762|TRPA1_HUMAN Transient receptor pote...,0.0050662435,False
2,>sp|P04629|NTRK1_HUMAN High affinity nerve gro...,0.010042973,False
3,>sp|Q9UI33|SCNBA_HUMAN Sodium channel protein ...,0.008308047,False
4,>sp|Q9Y5Y9|SCNAA_HUMAN Sodium channel protein ...,0.0052825967,False
...,...,...,...
283,>sp|P18509|PACA_HUMAN Pituitary adenylate cycl...,0.01865728,False
284,>sp|Q96LR4|TAFA4_HUMAN Chemokine-like protein ...,0.007869436,False
285,>sp|Q9BYJ1|LOXE3_HUMAN Hydroperoxide isomerase...,0.005664655,False
286,>sp|P01210|PENK_HUMAN Proenkephalin-A OS=Homo ...,0.008034096,False


The majority of proteins have been predicted as not transcription factors:

In [20]:
print(df.is_tf.mean().round(3)*100, "% of proteins have been predicted as transcription factors.", sep = "")

5.2% of proteins have been predicted as transcription factors.


We can complement our analysis with some data from UniProt. For each protein, we are downloading its function description:

In [15]:
import requests
from bs4 import BeautifulSoup as bs

df["uniprot_id"] = [v.split("|")[1] for v in df.seq_id.values.tolist()]

def get_descr(up_id):
  page = requests.get("https://www.uniprot.org/uniprot/" + up_id).text
  descr = bs(page, "html.parser").find("div", {"id": "function"}).p.text
  return descr

df["uniprot_description"] = [get_descr(r.uniprot_id) for i,r in df.iterrows()]
df

,seq_id,score,is_tf,uniprot_id,uniprot_description
0,>sp|Q15858|SCN9A_HUMAN Sodium channel protein ...,0.008337229,False,Q15858,Mediates the voltage-dependent sodium ion perm...
1,>sp|O75762|TRPA1_HUMAN Transient receptor pote...,0.0050662435,False,O75762,Receptor-activated non-selective cation channe...
2,>sp|P04629|NTRK1_HUMAN High affinity nerve gro...,0.010042973,False,P04629,Receptor tyrosine kinase involved in the devel...
3,>sp|Q9UI33|SCNBA_HUMAN Sodium channel protein ...,0.008308047,False,Q9UI33,This protein mediates the voltage-dependent so...
4,>sp|Q9Y5Y9|SCNAA_HUMAN Sodium channel protein ...,0.0052825967,False,Q9Y5Y9,Tetrodotoxin-resistant channel that mediates t...
...,...,...,...,...,...
283,>sp|P18509|PACA_HUMAN Pituitary adenylate cycl...,0.01865728,False,P18509,Binding to its receptor activates G proteins a...
284,>sp|Q96LR4|TAFA4_HUMAN Chemokine-like protein ...,0.007869436,False,Q96LR4,Modulates injury-induced and chemical pain hyp...
285,>sp|Q9BYJ1|LOXE3_HUMAN Hydroperoxide isomerase...,0.005664655,False,Q9BYJ1,Non-heme iron-containing lipoxygenase which is...
286,>sp|P01210|PENK_HUMAN Proenkephalin-A OS=Homo ...,0.008034096,False,P01210,Competes with and mimic the effects of opiate ...


For example, if we filter out all the proteins that have not been predicted as transcription factors:

In [ ]:
df[df.is_tf]

,seq_id,score,is_tf,uniprot_id,uniprot_description
15,>sp|Q9H4Q4|PRD12_HUMAN PR domain zinc finger p...,0.9780904,True,Q9H4Q4,Involved in the positive regulation of histone...
48,>sp|P37275|ZEB1_HUMAN Zinc finger E-box-bindin...,0.994453,True,P37275,Acts as a transcriptional repressor. Inhibits ...
56,>sp|Q8N1G0|ZN687_HUMAN Zinc finger protein 687...,0.8928037,True,Q8N1G0,May be involved in transcriptional regulation.
110,>sp|P78543|BTG2_HUMAN Protein BTG2 OS=Homo sap...,0.98999715,True,P78543,Anti-proliferative protein; the function is me...
135,>sp|A6NNA5|DRGX_HUMAN Dorsal root ganglia home...,0.99419653,True,A6NNA5,Transcription factor required for the formatio...
139,>sp|Q15306|IRF4_HUMAN Interferon regulatory fa...,0.9757959,True,Q15306,Transcriptional activator. Binds to the interf...
148,>sp|P15884|ITF2_HUMAN Transcription factor 4 O...,0.9588655,True,P15884,Transcription factor that binds to the immunog...
198,>sp|Q9BR39|JPH2_HUMAN Junctophilin-2 OS=Homo s...,0.97406197,True,Q9BR39,Membrane-binding protein that provides a struc...
204,>sp|O75398|DEAF1_HUMAN Deformed epidermal auto...,0.9718963,True,O75398,Transcription factor that binds to sequence wi...
210,>sp|P10588|NR2F6_HUMAN Nuclear receptor subfam...,0.9596225,True,P10588,Transcription factor predominantly involved in...


We see that in most of the predicted transcription factors, UniProt function description contains "Transcription factor" or similar terms.